<a href="https://colab.research.google.com/github/khalidmahamud/plant_disease_detection/blob/main/plant_disease_classification_v0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"khalidmahamud","key":"c8ab918e2693d125a7288c376d45b6c6"}'}

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [ ]:
!kaggle datasets download -d abdallahalidev/plantvillage-dataset

Dataset URL: https://www.kaggle.com/datasets/abdallahalidev/plantvillage-dataset
License(s): CC-BY-NC-SA-4.0
 99% 2.02G/2.04G [00:32<00:00, 129MB/s]
100% 2.04G/2.04G [00:32<00:00, 66.6MB/s]


In [ ]:
import zipfile

# Define the name of the downloaded zip file
zip_file_path = '/content/plantvillage-dataset.zip'  # Change this if the filename is different

# Create a directory to extract the files if needed
extract_dir = '/content/plantvillage-dataset'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [ ]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Set the path to where the dataset is extracted
dataset_path = '/content/plantvillage-dataset/plantvillage dataset/color'

tomato_directories = []

# List the tomato directories directly
for dir_name in os.listdir(dataset_path):
    if 'Tomato' in dir_name:
        tomato_directories.append(os.path.join(dataset_path, dir_name))

# Print the tomato directories
for directory in tomato_directories:
    print(directory)


/content/plantvillage-dataset/plantvillage dataset/color/Tomato___healthy
/content/plantvillage-dataset/plantvillage dataset/color/Tomato___Target_Spot
/content/plantvillage-dataset/plantvillage dataset/color/Tomato___Spider_mites Two-spotted_spider_mite
/content/plantvillage-dataset/plantvillage dataset/color/Tomato___Early_blight
/content/plantvillage-dataset/plantvillage dataset/color/Tomato___Tomato_Yellow_Leaf_Curl_Virus
/content/plantvillage-dataset/plantvillage dataset/color/Tomato___Leaf_Mold
/content/plantvillage-dataset/plantvillage dataset/color/Tomato___Tomato_mosaic_virus
/content/plantvillage-dataset/plantvillage dataset/color/Tomato___Bacterial_spot
/content/plantvillage-dataset/plantvillage dataset/color/Tomato___Septoria_leaf_spot
/content/plantvillage-dataset/plantvillage dataset/color/Tomato___Late_blight


In [ ]:
image_paths = []
labels = []

In [ ]:
# Define a mapping from directory names to labels
label_mapping = {
    "Tomato___healthy": 0,
    "Tomato___Late_blight": 1,
    "Tomato___Early_blight": 2,
    "Tomato___Septoria_leaf_spot": 3,
    "Tomato___Tomato_Yellow_Leaf_Curl_Virus": 4,
    "Tomato___Bacterial_spot": 5,
    "Tomato___Target_Spot": 6,
    "Tomato___Tomato_mosaic_virus": 7,
    "Tomato___Leaf_Mold": 8,
    "Tomato___Spider_mites Two-spotted_spider_mite": 9,
}

# Iterate through all the tomato directories
for directory in tomato_directories:
    # Get the label based on the directory name
    dir_name = os.path.basename(directory)  # Get the last part of the path
    label = label_mapping.get(dir_name, None)  # Get the corresponding label

    if label is not None:  # Proceed only if the label exists
        # Get all image files in the directory
        file_count = 0
        for file in os.listdir(directory):
            if file.lower().endswith(('.jpg', '.png', '.jpeg')):
                image_paths.append(os.path.join(directory, file))
                labels.append(str(label))  # Append the label
                file_count += 1
        print(f"Directory: {directory}, Added {file_count} files.")
    else:
        print(f"Warning: No label found for directory {dir_name}")

print(f"Total images processed: {len(image_paths)}")

Directory: /content/plantvillage-dataset/plantvillage dataset/color/Tomato___healthy, Added 1591 files.
Directory: /content/plantvillage-dataset/plantvillage dataset/color/Tomato___Target_Spot, Added 1404 files.
Directory: /content/plantvillage-dataset/plantvillage dataset/color/Tomato___Spider_mites Two-spotted_spider_mite, Added 1676 files.
Directory: /content/plantvillage-dataset/plantvillage dataset/color/Tomato___Early_blight, Added 1000 files.
Directory: /content/plantvillage-dataset/plantvillage dataset/color/Tomato___Tomato_Yellow_Leaf_Curl_Virus, Added 5357 files.
Directory: /content/plantvillage-dataset/plantvillage dataset/color/Tomato___Leaf_Mold, Added 952 files.
Directory: /content/plantvillage-dataset/plantvillage dataset/color/Tomato___Tomato_mosaic_virus, Added 373 files.
Directory: /content/plantvillage-dataset/plantvillage dataset/color/Tomato___Bacterial_spot, Added 2127 files.
Directory: /content/plantvillage-dataset/plantvillage dataset/color/Tomato___Septoria_lea

In [ ]:
# Create a DataFrame to store the image paths and their labels
df = pd.DataFrame({
    'image_path': image_paths,
    'label': labels
})

df.head(5)

,image_path,label
0,/content/plantvillage-dataset/plantvillage dat...,0
1,/content/plantvillage-dataset/plantvillage dat...,0
2,/content/plantvillage-dataset/plantvillage dat...,0
3,/content/plantvillage-dataset/plantvillage dat...,0
4,/content/plantvillage-dataset/plantvillage dat...,0


In [ ]:
print(df['label'].value_counts())  # Counts the number of images for each label

label
4    5357
5    2127
1    1909
3    1771
9    1676
0    1591
6    1404
2    1000
8     952
7     373
Name: count, dtype: int64


In [ ]:
# Define X and y
X = df['image_path']  # Your feature (image paths)
y = df['label']       # Your labels

In [ ]:
from sklearn.model_selection import train_test_split

# First, split into training+validation and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Then split the training+validation set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.2, stratify=y_train_val, random_state=42
)


# Create DataFrames for each split
df_train = pd.DataFrame({'image_path': X_train, 'label': y_train})
df_val = pd.DataFrame({'image_path': X_val, 'label': y_val})
df_test = pd.DataFrame({'image_path': X_test, 'label': y_test})

# Print the sample counts
print(f"Training samples: {len(df_train)}, Validation samples: {len(df_val)}, Test samples: {len(df_test)}")


Training samples: 11622, Validation samples: 2906, Test samples: 3632


In [ ]:
from imblearn.over_sampling import RandomOverSampler  # For oversampling

# Oversampling to handle class imbalance in training data
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(np.array(X_train).reshape(-1, 1), y_train)

# Create a new DataFrame with resampled data
df_train_resampled = pd.DataFrame({
    'image_path': X_train_resampled.flatten(),
    'label': y_train_resampled
})

In [ ]:
# Check class distribution in each set
train_class_distribution = y_train_resampled.value_counts()
val_class_distribution = y_val.value_counts()
test_class_distribution = y_test.value_counts()

print("Training Class Distribution:\n", train_class_distribution)
print("\nValidation Class Distribution:\n", val_class_distribution)
print("\nTest Class Distribution:\n", test_class_distribution)

Training Class Distribution:
 label
0    3429
5    3429
4    3429
1    3429
9    3429
6    3429
8    3429
7    3429
3    3429
2    3429
Name: count, dtype: int64

Validation Class Distribution:
 label
4    857
5    341
1    305
3    283
9    268
0    255
6    225
2    160
8    152
7     60
Name: count, dtype: int64

Test Class Distribution:
 label
4    1071
5     425
1     382
3     354
9     335
0     318
6     281
2     200
8     191
7      75
Name: count, dtype: int64


In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# Calculate class weights for the training set
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_resampled), y=y_train_resampled)
class_weights = dict(enumerate(class_weights))

print("Class Weights:", class_weights)

Class Weights: {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 6: 1.0, 7: 1.0, 8: 1.0, 9: 1.0}


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define your data augmentation for the training set
data_augmentation = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create a training generator with augmentation
train_generator = data_augmentation.flow_from_dataframe(
    dataframe=df_train_resampled,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse'
)

# Create a validation generator (without augmentation)
val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=df_val,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse'
)

# Create a test generator (without augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    shuffle=False
)


Found 34290 validated image filenames belonging to 10 classes.
Found 2906 validated image filenames belonging to 10 classes.
Found 3632 validated image filenames belonging to 10 classes.


In [ ]:
from tensorflow.keras.applications import ResNet50, InceptionV3  # Import ResNet50 along with InceptionV3
from tensorflow.keras.layers import Input, Concatenate, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Dropout

# Load pre-trained ResNet50
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet_base.trainable = False  # Freeze ResNet50 layers

# Load pre-trained InceptionV3
inception_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
inception_base.trainable = False  # Freeze InceptionV3 layers

# Define the input
input_layer = Input(shape=(224, 224, 3))

# Extract features from ResNet50 and InceptionV3
resnet_features = resnet_base(input_layer)
resnet_features = GlobalAveragePooling2D()(resnet_features)

inception_features = inception_base(input_layer)
inception_features = GlobalAveragePooling2D()(inception_features)

# Concatenate the features
combined_features = Concatenate()([resnet_features, inception_features])

# Add a fully connected layer
fc_layer = Dense(128, activation='relu')(combined_features)

# Add dropout for regularization
dropout_layer = Dropout(0.5)(fc_layer)  # 50% dropout rate

# Output layer
output_layer = Dense(len(np.unique(y_train_resampled)), activation='softmax')(dropout_layer)

# Build the model
ensemble_model = Model(inputs=input_layer, outputs=output_layer)

# Unfreeze the last few layers of ResNet50 and InceptionV3
# for layer in resnet_base.layers[-10:]:
#     layer.trainable = True

# for layer in inception_base.layers[-10:]:
#     layer.trainable = True

# Compile the model with class weights
fine_tune_learning_rate = 1e-5
ensemble_model.compile(optimizer=Adam(learning_rate=fine_tune_learning_rate),
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])

ensemble_model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ resnet50 (Functional)     │ (None, 7, 7, 2048)     │     23,587,712 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ inception_v3 (Functional) │ (None, 5, 5, 2048)     │     21,802,784 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d  │ (None, 2048)           │              0 │ resnet50[0][0]         │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 2048)           │              0 │ inception_v3[0][0]     │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_2             │ (None, 4096)           │              0 │ global_average_poolin… │
│ (Concatenate)             │                        │                │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │        524,416 │ concatenate_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 128)            │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 10)             │          1,290 │ dropout[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 45,916,202 (175.16 MB)

 Trainable params: 525,706 (2.01 MB)

 Non-trainable params: 45,390,496 (173.15 MB)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Define the checkpoint for val_loss
checkpoint_loss = ModelCheckpoint('best_model_loss.keras', monitor='val_loss', save_best_only=True, verbose=1)

# Define the checkpoint for val_accuracy
checkpoint_accuracy = ModelCheckpoint('best_model_accuracy.keras', monitor='val_accuracy', save_best_only=True, verbose=1)

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)  # Stop after 5 epochs of no improvement

# Train with callbacks monitoring both val_loss and val_accuracy
history_fine_tune = ensemble_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    class_weight=class_weights,  # Use class weights to balance the classes
    callbacks=[checkpoint_loss, checkpoint_accuracy, early_stopping]
)

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 480/1072 ━━━━━━━━━━━━━━━━━━━━ 1:44:09 11s/step - accuracy: 0.1284 - loss: 2.6668

In [ ]:
import matplotlib.pyplot as plt

# Plot the training history
plt.plot(history_fine_tune.history['accuracy'], label='Train Accuracy')
plt.plot(history_fine_tune.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Extracting accuracy and loss from the history
epochs = range(1, len(history_fine_tune.history['accuracy']) + 1)
train_accuracy = history_fine_tune.history['accuracy']
val_accuracy = history_fine_tune.history['val_accuracy']
train_loss = history_fine_tune.history['loss']
val_loss = history_fine_tune.history['val_loss']

# Plotting accuracy
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_accuracy, label='Training Accuracy', marker='o')
plt.plot(epochs, val_accuracy, label='Validation Accuracy', marker='o')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()

# Plotting loss
plt.subplot(1, 2, 2)
plt.plot(epochs, train_loss, label='Training Loss', marker='o')
plt.plot(epochs, val_loss, label='Validation Loss', marker='o')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()


In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = ensemble_model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy:.4f}, Test loss: {test_loss:.4f}")

In [ ]:
from sklearn.metrics import classification_report

# Get predictions
y_pred = ensemble_model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

# Print classification report
y_test_numeric = np.array([int(label) for label in y_test])

# Print classification report
print(classification_report(y_test_numeric, y_pred_classes))


In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Define class names
class_names = [
    "Tomato - Healthy",
    "Tomato - Late Blight",
    "Tomato - Early Blight",
    "Tomato - Septoria Leaf Spot",
    "Tomato - Yellow Leaf Curl Virus",
    "Tomato - Bacterial Spot",
    "Tomato - Target Spot",
    "Tomato - Mosaic Virus",
    "Tomato - Leaf Mold",
    "Tomato - Spider Mites"
]

# Compute confusion matrix
cm = confusion_matrix(y_test_numeric, y_pred_classes)

# Visualize confusion matrix
plt.figure(figsize=(12, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Print classification report
report = classification_report(y_test_numeric, y_pred_classes, target_names=class_names)
print(report)
